# 💳 Credit Card Fraud Detection

---

**Author:** Piyush Ramteke  
**Program:** CodSoft Data Science Internship  
**Task:** Task 3 - Credit Card Fraud Detection

---

## 🚨 Problem Statement

Credit card fraud causes billions of dollars in losses every year. As digital payments grow, so does fraudulent activity. The challenge is to **automatically detect fraudulent transactions** among hundreds of thousands of legitimate ones.

This is a **binary classification** problem:
- **Class 0** 🟢 → Genuine transaction
- **Class 1** 🔴 → Fraudulent transaction

The key difficulty is **class imbalance** — frauds make up less than 0.2% of all transactions. A naive model that predicts everything as "genuine" would get 99.8% accuracy but catch zero frauds.

**Goal:** Build a model that maximizes **Recall** (catching as many frauds as possible) while maintaining acceptable **Precision** (minimizing false alarms). We will use **Interactive Visualizations** to explore this imbalance deeply.

---
## 1️⃣ Importing Libraries 📦

In [ ]:
# Core Libraries
import numpy as np
import pandas as pd
import warnings

# Interactive Visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Interactive Widgets
from ipywidgets import interact, widgets

# Modeling & Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve
)
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')
print('✅ All libraries loaded with Interactive Capabilities! 🚀')

---
## 2️⃣ Data Loading & Exploration 📂

In [ ]:
df = pd.read_csv('creditcard.csv')
print(f'📐 Shape: {df.shape}')
df.head()

---
## 3️⃣ Interactive EDA 🔍

In [ ]:
# ── 3.1 Interactive Class Distribution ────────────────────────

counts = df['Class'].value_counts().reset_index()
counts.columns = ['Class', 'Count']
counts['Label'] = counts['Class'].map({0: 'Genuine 🟢', 1: 'Fraud 🔴'})

fig = px.pie(counts, values='Count', names='Label', 
             title='📊 Class Imbalance Distribution',
             color='Label',
             color_discrete_map={'Genuine 🟢': '#00CC96', 'Fraud 🔴': '#EF553B'},
             hole=0.4)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

**💡 Interpretation:** The dataset is **extremely imbalanced** (only ~0.17% fraud). Interactive charts help visualize this huge disparity clearly.

In [ ]:
# ── 3.2 Feature Distribution Explorer (Interactive Widget) ────

print("👇 Explore distributions of different V-features by Class")
@interact(Feature=['V1', 'V2', 'V3', 'V4', 'V5', 'V10', 'V14', 'Amount'])
def plot_feature_dist(Feature):
    fig = px.histogram(df, x=Feature, color='Class',
                       nbins=50,
                       title=f'Distribution of {Feature} by Class',
                       barmode='overlay',
                       opacity=0.7,
                       color_discrete_map={0: '#00CC96', 1: '#EF553B'},
                       labels={'Class': 'Transaction Type'})
    fig.show()

In [ ]:
# ── 3.3 Transaction Amount vs Time (Interactive) ──────────────

fig = px.scatter(df.sample(2000), x='Time', y='Amount', color='Class',
                 title='⏱️ Transaction Amount vs. Time (Sampled)',
                 color_continuous_scale=['#00CC96', '#EF553B'],
                 size='Amount', size_max=20,
                 hover_data=['V1', 'V2'])
fig.show()

In [ ]:
# ── 3.4 Interactive Correlation Heatmap ───────────────────────

corr = df.corr()

fig = px.imshow(corr, text_auto=False, aspect="auto",
                title='🔥 Feature Correlation Matrix',
                color_continuous_scale='RdBu_r',
                origin='lower')
fig.show()

---
## 4️⃣ Data Preprocessing 🧹

We need to scale the `Amount` and `Time` columns as other `V` features are already scaled.

In [ ]:
scaler = StandardScaler()
df['Amount_Scaled'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df['Time_Scaled'] = scaler.fit_transform(df['Time'].values.reshape(-1, 1))

df.drop(['Time', 'Amount'], axis=1, inplace=True)

X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print('✅ Data Scaled & Split Successfully!')

---
## 5️⃣ Handling Imbalance (SMOTE) ⚖️

In [ ]:
print('⏳ Applying SMOTE... (This might act slow mostly because resizing huge datasets)')
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f'✅ Resampled Shape: {X_train_resampled.shape}')
print(f'✅ Genuine count: {sum(y_train_resampled == 0)}')
print(f'✅ Fraud count:   {sum(y_train_resampled == 1)}')

---
## 6️⃣ Model Training & Evaluation 🤖

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_resampled, y_train_resampled)

y_pred = model.predict(X_test)

print('✅ Model Trained!')

In [ ]:
# ── 6.1 Interactive Confusion Matrix ──────────────────────────

cm = confusion_matrix(y_test, y_pred)

# Create annotated heatmap
fig = ff.create_annotated_heatmap(z=cm, x=['Predicted Genuine', 'Predicted Fraud'], 
                                  y=['Actual Genuine', 'Actual Fraud'], 
                                  colorscale='Blues', showscale=True)

fig.update_layout(title_text='📉 Confusion Matrix')
fig.show()

print('\n📄 Classification Report:\n')
print(classification_report(y_test, y_pred))

### 🎉 Conclusion
We successfully:
1. Explored the data using **Interactive Plots**.
2. Handled class imbalance using **SMOTE**.
3. Trained a Logistic Regression model.
4. Visualized results with an **Interactive Confusion Matrix**.